In [ ]:
from glob import glob
import h5py
import os
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
from matplotlib import rcParams
import matplotlib as mpl
rcParams['figure.figsize'] = [15,10]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 80

COLOR = 'white'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

In [ ]:
def pca_func(flatten_final,n_comp,ncols=10):
    
    pca = PCA(n_components=n_comp)
    lower_dimensional_data = pca.fit_transform(flatten_final)
    approximation = pca.inverse_transform(lower_dimensional_data)
    #print(pca.n_components)
    
    return approximation,lower_dimensional_data
    

In [ ]:
def visualize_pca(flatten_final, approximation, ncols=10):
    substract = np.abs(flatten_final - approximation)
    nsample = flatten_final.shape[0]
    #subs_mean = np.mean(substract,axis=0)
    
    _, axes = plt.subplots(nrows=3,ncols=ncols,figsize=(20,8))
    
    for idx in range(ncols):
        
        rand_num = np.random.randint(0,nsample)
        #print(rand_num)
        
        
        axes[0][idx].imshow(flatten_final[rand_num].reshape(320, 320),
              cmap = plt.cm.gray, interpolation='nearest',
              clim=(0, 255));
        axes[1][idx].imshow(approximation[rand_num].reshape(320, 320),
              cmap = plt.cm.gray, interpolation='nearest',
              clim=(0, 255));
    
        axes[2][idx].imshow(np.sinh(substract[rand_num]).reshape(320, 320),
              cmap = plt.cm.gray, interpolation='nearest',
              clim=(0, 255));
        
        if idx == 0:
            
            axes[0][idx].set_ylabel('Original',fontsize=10,fontweight='bold')
            axes[1][idx].set_ylabel('Reconstructed',fontsize=10,fontweight='bold')
            axes[2][idx].set_ylabel('Subtracted',fontsize=10,fontweight='bold')
            
        axes[0][idx].set_yticks([])
        axes[0][idx].set_xticks([])
        
        axes[1][idx].set_yticks([])
        axes[1][idx].set_xticks([])
        
        axes[2][idx].set_yticks([])
        axes[2][idx].set_xticks([])
        
    plt.subplots_adjust(wspace=0,hspace=0)
    _.patch.set_facecolor('#423f3b')
    plt.show()
    

In [ ]:
def visualize_latent(low_dim_data,dimension):
    
    
    if dimension == 2:
    
        ((pc1_min, pc2_min), 
        (pc1_max, pc2_max)) = np.percentile(low_dim_data, q=[5, 95], axis=0)


        roi_rect = patches.Rectangle(xy=(pc1_min, pc2_min),
                                 width=pc1_max-pc1_min,
                                 height=pc2_max-pc2_min, alpha=.4)

        fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(15,15))

        ax.scatter(*low_dim_data.T)
        ax.add_patch(roi_rect)
        ax.set_xlabel('$pc_1$')
        ax.set_ylabel('$pc_2$')
        fig.patch.set_facecolor('#423f3b')

        plt.show()
        
    if dimension == 3:
                
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        ax.scatter(*low_dim_data.T)    
        ax.set_xlabel('$pc_1$')
        ax.set_ylabel('$pc_2$')
        ax.set_zlabel('$pc_3$')
        fig.patch.set_facecolor('#423f3b')
        plt.show()


## Data loading

In [ ]:
directory = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/'

In [ ]:
h5_files = glob(os.path.join(directory,'**/*.h5'))

In [ ]:
h5_files

In [ ]:
data_1441 = h5py.File(h5_files[0],'r')
data_1386 = h5py.File(h5_files[1],'r')

## 1386

In [ ]:
keys_1386 = [x for x in data_1386.keys()]

In [ ]:
final_1386 = np.concatenate((np.array(data_1386[keys_1386[0]]),np.array(data_1386[keys_1386[1]])))

for i in range(len(keys_1386)-2):
    
    final_1386 = np.concatenate((final_1386,np.array(data_1386[keys_1386[i+2]])))
    
final_1386.shape

In [ ]:
flatten_final_1386 = np.resize(final_1386, (final_1386.shape[0],320*320))
flatten_final_1386.shape

In [ ]:
approx_1386,low_dim_1386 = pca_func(flatten_final_1386,3)

In [ ]:
visualize_pca(flatten_final_1386, approx_1386)

In [ ]:
visualize_latent(low_dim_1386,3)

## 1441

In [ ]:
keys_1441 = [x for x in data_1441.keys()]

In [ ]:
f = np.concatenate((np.array(data_1441[keys_1441[0]]),np.array(data_1441[keys_1441[1]])))
final_1441 = np.concatenate((f,np.array(data_1441[keys_1441[2]])))

In [ ]:
flatten_final_1441 = np.resize(final_1441, (final_1441.shape[0],320*320))
flatten_final_1441.shape

In [ ]:
approx_1441,low_dim_1441 = pca_func(flatten_final_1441,3)

In [ ]:
visualize_pca(flatten_final_1441, approx_1441)

In [ ]:
visualize_latent(low_dim_1441,3)

Now we can try pca for every psfstack in 1441. We will calculate one by one.

In [ ]:
batch1 = np.array(data_1441[keys_1441[0]])
batch2 = np.array(data_1441[keys_1441[1]])
batch3 = np.array(data_1441[keys_1441[2]])

In [ ]:
flatten_batch1 = np.resize(batch1, (batch1.shape[0],320*320))
flatten_batch2 = np.resize(batch2, (batch2.shape[0],320*320))
flatten_batch3 = np.resize(batch3, (batch3.shape[0],320*320))

In [ ]:
approx1_1441,low_dim_1441_1 =  pca_func(flatten_batch1,3)
approx2_1441,low_dim_1441_2 =  pca_func(flatten_batch2,3)
approx3_1441,low_dim_1441_3 =  pca_func(flatten_batch3,3)

In [ ]:
visualize_pca(flatten_batch1, approx1_1441)

In [ ]:
visualize_latent(low_dim_1441_1,3)

In [ ]:
visualize_pca(flatten_batch2, approx2_1441)

In [ ]:
visualize_latent(low_dim_1441_2,3)

In [ ]:
visualize_pca(flatten_batch3, approx3_1441)

In [ ]:
visualize_latent(low_dim_1441_3,3)